# MLP

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

Using TensorFlow backend.


In [2]:
def data():
    X_train = np.load("X_train.npy")
    y_train = np.load("y_train.npy")
    X_val = np.load("X_val.npy")
    y_val = np.load("y_val.npy")
    
    return X_train, y_train, X_val, y_val

In [3]:
def model(X_train, y_train, X_val, y_val):
    model = Sequential()
    model.add(Dense({{choice([128, 256, 512, 1024])}}, input_shape=(243,)))
    model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([128, 256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    
    if conditional({{choice(['two', 'three'])}}) == 'three':
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    adam = Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
    
    choice_optimizer = {{choice(['adam', 'rmsprop', 'sgd'])}}
    if choice_optimizer == 'adam':
        optimizer = adam
    elif choice_optimizer == 'rmsprop':
        optimizer = rmsprop
    elif choice_optimizer == 'sgd':
        optimizer = sgd
        
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(X_train, y_train,
              batch_size={{choice([128, 256, 512])}},
              epochs=100,
              verbose=1,
              validation_data=(X_val, y_val))
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy: {}'.format(acc))
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [3]:
X_train, y_train, X_val, y_val = data()

In [ ]:
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=50,
                                      trials=Trials(),
                                      notebook_name='models')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Activation, Dropout
except:
    pass

try:
    from tensorflow.keras.callbacks import EarlyStopping
except:
    pass

try:
    from tensorflow.keras.optimizers import Adam, RMSprop, SGD
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [128, 256, 512, 1024]),
        'Activation': hp.choice('Activation', ['relu', 'sigmoid', 'tanh']),
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense_1': hp.choice('Dense_1', [128, 256, 512, 1024]),
        'Activation_1': hp.choice('Acti

# MLP 2

In [5]:
from neuro_evolution.evolution import NeuroEvolution

params = {
    "epochs": [10, 20, 50],
    "batch_size": [128, 256, 512],
    "n_layers": [1, 2, 3, 4],
    "n_neurons": [128, 256, 512, 1024],
    "dropout": [0.1, 0.2, 0.5],
    "optimizers": ["rmsprop", "adam", "sgd"],
    "activations": ["relu", "sigmoid", "tanh"],
    "last_layer_activations": ["sigmoid"],
    "losses": ["binary_crossentropy"],
    "metrics": ["accuracy"]
}

In [6]:
search = NeuroEvolution(generations = 10, population = 10, params=params)

In [7]:
search.evolve(X_train, y_train, X_val, y_val)

100%|██████████| 10/10 [29:25<00:00, 164.05s/it]

[06/Nov/2018 00:04:24] INFO - best accuracy: 0.6453234025862861, best params: {'epochs': 20, 'batch_size': 512, 'n_layers': 2, 'n_neurons': 128, 'dropout': 0.5, 'optimizers': 'adam', 'activations': 'sigmoid', 'last_layer_activations': 'sigmoid', 'losses': 'binary_crossentropy', 'metrics': 'accuracy'}


In [8]:
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=243))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

Train on 255098 samples, validate on 127550 samples
Epoch 1/100
255098/255098 [==============================] - 215s 843us/step - loss: 0.6579 - acc: 0.6213 - val_loss: 0.6509 - val_acc: 0.6358
Epoch 2/100
 93184/255098 [=========>....................] - ETA: 2:12 - loss: 0.6561 - acc: 0.6299

KeyboardInterrupt: 